In [1]:
import os
import torch
from torch import nn
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torch.utils.data import DataLoader

from apn import APN
from data.cub import CUBDatasetSimple

In [2]:
test_transforms = T.Compose([
    T.Resize(size=448, antialias=True),
    T.CenterCrop(size=448),
    T.ToTensor()
])

dataset_val = CUBDatasetSimple(os.path.join('datasets', 'CUB'), split='val', transforms=test_transforms)

In [3]:
net = APN(attr_class_map=dataset_val.attr_class_map, k=107, attr_groups=dataset_val.attr_groups)
state_dict = torch.load('checkpoints/checkpoint.pt', map_location='cpu')
net.load_state_dict(state_dict)

In [202]:
idx = 5540
net.eval()
with torch.no_grad():
    image, class_tgts, attr_tgts = dataset_val[idx]
    image = image.unsqueeze(0)
    final_logits, local_logits, attn_maps, prototypes, max_logit_coords = net(image)

In [203]:
class_tgts

In [204]:
attn_maps.shape

In [205]:
dataset_val.attribute_df.reset_index(drop=True)

In [206]:
vals, idxs = torch.topk(dataset_val.attr_class_map[190], k=10)
vals, idxs

In [207]:
torch.max(attn_maps.squeeze())

In [200]:
dataset_val.attribute_df.reset_index(drop=True).iloc[idxs.numpy()]

In [188]:
plt.imshow(image.squeeze().permute(1,2,0).numpy())
plt.show()